In [ ]:

# Re-import libraries after reset
import pandas as pd
from collections import defaultdict
from bisect import bisect_right
from functions import stop_templates
import numpy as np

file = #insert file of choice

df = pd.read_csv(file, parse_dates=["DateTime"])



In [3]:
def print_station(df, station, service_day):
    # Filter by station and service day
    day_data = df[(df["Origin"] == station) & (df["ServiceDay"] == service_day)]

    if day_data.empty:
        print(f"No data for station {station} on {service_day}")
        return

    # Group by journey ID and sort journeys by their first DateTime
    journeys = (
        day_data.groupby("StationJourneyID")
        .apply(lambda g: g.sort_values("DateTime"))
        .reset_index(drop=True)
    )
    
    # Sort by first DateTime of each journey
    first_times = (
        journeys.groupby("StationJourneyID")["DateTime"]
        .min()
        .sort_values()
        .index.tolist()
    )

    for journey_id in first_times:
        group = journeys[journeys["StationJourneyID"] == journey_id]
        print(f"\nStationJourney {int(journey_id)} at {station} on {service_day}")
        for _, row in group.iterrows():
            print(f"  {row['DateTime']}  |  Due in: {row['Minutes']} min | Destination: {row['Destination']}")

def print_journey(df, journey_id, stop_templates):
    journey_df = df[df["TramJourneyID"] == journey_id].copy()

    if journey_df.empty:
        print(f"No {journey_id}")
        return

    # Attempt to extract (origin, destination) from journey ID using known templates
    def find_route_from_id(jid):
        for (start, end) in stop_templates:
            if start in jid and end in jid and jid.index(start) < jid.index(end):
                return start, end
        return None, None

    origin, destination = find_route_from_id(journey_id)

    if origin and destination and (origin, destination) in stop_templates:
        stop_order = stop_templates[(origin, destination)]
    else:
        print(f"No stop template found\n")
        stop_order = sorted(journey_df["Origin"].unique())

    print(f"TramJourney {journey_id} ({origin or '???'} → {destination or '???'})\n")

    for stop in stop_order:
        stop_logs = journey_df[journey_df["Origin"] == stop].sort_values("DateTime")
        if stop_logs.empty:
            continue
        print(f"{stop}")
        for _, row in stop_logs.iterrows():
            dt = row["DateTime"].strftime("%Y-%m-%d %H:%M:%S")
            due = f"{row['Minutes']:.1f} min"
            print(f"  {dt} | Due in: {due}")
        print()


In [4]:
import pandas as pd
from datetime import timedelta

def stitch_forecasts_by_station(df, max_gap_minutes=8, min_logs=5):
    df = df.sort_values(['Origin', 'ServiceDay', 'Destination', 'DateTime']).reset_index(drop=True)
    df['StationJourneyID'] = pd.NA

    journey_id = 1
    grouped = df.groupby(['Origin', 'ServiceDay', 'Destination'])

    for (station, day, dest), group in grouped:
        group = group.sort_values("DateTime")
        active_journeys = []
        journey_meta = []
        journey_timestamps = []
        journey_min_minutes = []

        for idx, row in group.iterrows():
            assigned = False
            to_remove = []

            for i in range(len(active_journeys)):
                last_idx = active_journeys[i][-1]
                last_row = group.loc[last_idx]

                time_diff = (row['DateTime'] - last_row['DateTime']).total_seconds() / 60
                minute_diff = last_row['Minutes'] - row['Minutes'] if pd.notna(row['Minutes']) and pd.notna(last_row['Minutes']) else 0
                forecast_jump = row['Minutes'] - last_row['Minutes'] if pd.notna(row['Minutes']) and pd.notna(last_row['Minutes']) else 0
                min_seen = journey_min_minutes[i]

                # Prevent duplicate timestamp inclusion
                if row['DateTime'] in journey_timestamps[i]:
                    continue

                # Allow continuation if time and countdown flow logically
                if (
                    0 <= time_diff <= max_gap_minutes and
                    -2 <= minute_diff <= 5 and  # Allow small reset tolerance
                    forecast_jump <= 3 and     # Allow for slight noise
                    not (min_seen <= 1 and forecast_jump > 3)  # Avoid real resets only if journey looks done
                ):
                    active_journeys[i].append(idx)
                    journey_meta[i] = row
                    journey_timestamps[i].add(row['DateTime'])
                    journey_min_minutes[i] = min(min_seen, row['Minutes']) if pd.notna(row['Minutes']) else min_seen
                    assigned = True
                    break

                # Mark to finalize if it's too old or clearly done
                elif time_diff > max_gap_minutes or (min_seen <= 1 and forecast_jump > 3):
                    to_remove.append(i)

            # Finalize any closed journeys
            for i in sorted(to_remove, reverse=True):
                journey = active_journeys.pop(i)
                if len(journey) >= min_logs:
                    for row_idx in journey:
                        df.at[row_idx, 'StationJourneyID'] = journey_id
                    journey_id += 1
                journey_meta.pop(i)
                journey_timestamps.pop(i)
                journey_min_minutes.pop(i)

            # Start new journey if not assigned
            if not assigned:
                active_journeys.append([idx])
                journey_meta.append(row)
                journey_timestamps.append({row['DateTime']})
                journey_min_minutes.append(row['Minutes'] if pd.notna(row['Minutes']) else float('inf'))

        # Finalize remaining journeys
        for journey in active_journeys:
            if len(journey) >= min_logs:
                for row_idx in journey:
                    df.at[row_idx, 'StationJourneyID'] = journey_id
                journey_id += 1

    return df


In [5]:
station_logs = stitch_forecasts_by_station(df)

In [6]:
from collections import defaultdict
import pandas as pd

def build_clean_tram_journeys(df, stop_templates, max_gap_min=6, allow_missing=1, min_stops=6, verbose=True):
    df = df.copy()
    df["EstimatedArrival"] = df["DateTime"] + pd.to_timedelta(df["Minutes"], unit="m")
    df["ServiceDate"] = df["DateTime"].dt.date

    # Step 1: Group logs by StationJourneyID and extract metadata
    grouped = df.groupby("StationJourneyID")
    sjid_info = []
    for sjid, group in grouped:
        dest = group["Destination"].iloc[0]
        station = group["Origin"].iloc[0]
        eta = group["EstimatedArrival"].max()
        date = group["ServiceDate"].iloc[0]
        sjid_info.append({
            "StationJourneyID": sjid,
            "Station": station,
            "Destination": dest,
            "EstimatedArrival": eta,
            "ServiceDate": date
        })

    sjid_df = pd.DataFrame(sjid_info)
    used_ids = set()
    journeys = []
    journey_counts = defaultdict(lambda: defaultdict(int))  # (origin, dest) → date → counter

    # Step 2: Loop over templates
    for (start, end), template in stop_templates.items():
        for date in sjid_df["ServiceDate"].unique():
            candidates = sjid_df[
                (sjid_df["Destination"] == end) &
                (sjid_df["ServiceDate"] == date)
            ]

            starters = candidates[candidates["Station"] == start].sort_values("EstimatedArrival")

            for _, start_row in starters.iterrows():
                if start_row["StationJourneyID"] in used_ids:
                    continue

                block = [start_row["StationJourneyID"]]
                current_time = start_row["EstimatedArrival"]
                missing = 0
                valid = True

                for stop in template[1:]:
                    possible = candidates[
                        (candidates["Station"] == stop) &
                        (~candidates["StationJourneyID"].isin(used_ids))
                    ]
                    possible = possible[possible["EstimatedArrival"].between(
                        current_time, current_time + pd.Timedelta(minutes=max_gap_min)
                    )].sort_values("EstimatedArrival")

                    if possible.empty:
                        missing += 1
                        if missing > allow_missing:
                            valid = False
                            break
                    else:
                        next_row = possible.iloc[0]
                        block.append(next_row["StationJourneyID"])
                        current_time = next_row["EstimatedArrival"]

                if valid and len(block) >= min_stops:
                    journey_counts[(start, end)][date] += 1
                    jid = f"{start}_{end}{journey_counts[(start, end)][date]:02d}_{date.year}_{date.month:02d}_{date.day:02d}"
                    matched_rows = df[df["StationJourneyID"].isin(block)].copy()
                    matched_rows["TramJourneyID"] = jid
                    journeys.append(matched_rows)
                    used_ids.update(block)

                    if verbose:
                        print(f"Built journey {jid} with {len(block)} stops")

    if not journeys:
        return pd.DataFrame()

    return pd.concat(journeys, ignore_index=True)


In [ ]:
full_journeys = build_clean_tram_journeys(station_logs, stop_templates)

Built journey BRI_BRO01_2020_01_20 with 31 stops
Built journey BRI_BRO02_2020_01_20 with 31 stops
Built journey BRI_BRO03_2020_01_20 with 31 stops
Built journey BRI_BRO04_2020_01_20 with 31 stops
Built journey BRI_BRO05_2020_01_20 with 31 stops
Built journey BRI_BRO06_2020_01_20 with 31 stops
Built journey BRI_BRO07_2020_01_20 with 31 stops
Built journey BRI_BRO08_2020_01_20 with 31 stops
Built journey BRI_BRO09_2020_01_20 with 31 stops
Built journey BRI_BRO10_2020_01_20 with 31 stops
Built journey BRI_BRO11_2020_01_20 with 31 stops
Built journey BRI_BRO12_2020_01_20 with 31 stops
Built journey BRI_BRO13_2020_01_20 with 31 stops
Built journey BRI_BRO01_2020_01_21 with 31 stops
Built journey BRI_BRO02_2020_01_21 with 31 stops
Built journey BRI_BRO03_2020_01_21 with 31 stops
Built journey BRI_BRO04_2020_01_21 with 31 stops
Built journey BRI_BRO05_2020_01_21 with 31 stops
Built journey BRI_BRO06_2020_01_21 with 31 stops
Built journey BRI_BRO07_2020_01_21 with 31 stops
Built journey BRI_BR

In [ ]:
print_journey(full_journeys, "BRI_BRO01_2020_01_20", stop_templates)

🚋 TramJourney BRI_BRO01_2020_01_20 (BRI → BRO)

📍 BRI
  2020-01-20 05:12:02 | Due in: 17.0 min
  2020-01-20 05:14:02 | Due in: 15.0 min
  2020-01-20 05:16:05 | Due in: 13.0 min
  2020-01-20 05:18:07 | Due in: 11.0 min
  2020-01-20 05:20:02 | Due in: 9.0 min
  2020-01-20 05:22:02 | Due in: 7.0 min
  2020-01-20 05:24:05 | Due in: 5.0 min
  2020-01-20 05:26:05 | Due in: 3.0 min
  2020-01-20 05:28:05 | Due in: 1.0 min
  2020-01-20 05:30:02 | Due in: 0.0 min

📍 CHE
  2020-01-20 05:12:05 | Due in: 18.0 min
  2020-01-20 05:14:04 | Due in: 16.0 min
  2020-01-20 05:16:05 | Due in: 14.0 min
  2020-01-20 05:18:07 | Due in: 12.0 min
  2020-01-20 05:20:05 | Due in: 10.0 min
  2020-01-20 05:22:04 | Due in: 8.0 min
  2020-01-20 05:24:05 | Due in: 6.0 min
  2020-01-20 05:26:05 | Due in: 4.0 min
  2020-01-20 05:28:05 | Due in: 2.0 min
  2020-01-20 05:30:05 | Due in: 0.0 min

📍 LAU
  2020-01-20 05:14:04 | Due in: 18.0 min
  2020-01-20 05:16:05 | Due in: 16.0 min
  2020-01-20 05:18:07 | Due in: 14.0 min


In [ ]:
def journey_stats(df):
    """
    Print basic statistics for tram journeys
    """
    if not {'TramJourneyID', 'DateTime', 'Origin', 'Destination'}.issubset(df.columns):
        print("Required columns not found in the DataFrame.")
        return

    df = df.copy()
    df["DateTime"] = pd.to_datetime(df["DateTime"])

    # Compute journey durations
    journey_times = df.groupby("TramJourneyID")["DateTime"].agg(["min", "max"])
    journey_times["duration_min"] = (journey_times["max"] - journey_times["min"]).dt.total_seconds() / 60

    print("Journey Duration")
    print(journey_times["duration_min"].describe())
    print()

    # Number of unique stops per journey
    stops_per_journey = df.groupby("TramJourneyID")["Origin"].nunique()
    print("Stops per Journey")
    print(stops_per_journey.describe())
    print()

    # Most common Origin → Destination routes
    routes = df.groupby("TramJourneyID").agg({"Destination": "first"})
    route_counts = routes.groupby([ "Destination"]).size().sort_values(ascending=False)
    print("Destination Counts")
    print(route_counts.head(10))


In [ ]:
stats = journey_stats(full_journeys)
stats

Journey Duration
count    4954.000000
mean       67.292871
std         7.730042
min        26.016667
25%        63.983333
50%        67.950000
75%        71.945833
max       130.000000
Name: duration_min, dtype: float64

Stops per Journey
count    4954.000000
mean       23.419257
std         3.478532
min         7.000000
25%        22.000000
50%        22.000000
75%        25.000000
max        31.000000
Name: Origin, dtype: float64

Destination Counts
Destination
TAL    823
BRI    804
PAR    701
BRO    634
SAN    634
TPT    525
SAG    506
CON    297
HEU     18
RED     12
dtype: int64


In [ ]:
journey_lengths = df.groupby("TramJourneyID").size()
print(journey_lengths.describe())
print(journey_lengths.sample(5))


count    23850.000000
mean       239.537442
std         42.687743
min         63.000000
25%        217.000000
50%        231.000000
75%        260.000000
max        676.000000
dtype: float64
TramJourneyID
TPT_TAL22_2020_02_22    219
BRO_BRI04_2020_03_03    304
BRO_BRI13_2020_02_11    306
SAN_PAR06_2020_03_18    160
SAN_BRO23_2020_02_29    216
dtype: int64
